In [14]:
import pandas as pd

# Load the dataset
file_path = r'C:\Users\wziller\Milwaukee Tool\_Global AME\MSOE\CS5610\M6\csc5610base\GroupProjectM6\Awards_Batting_HoF.csv'
fe_batting = pd.read_csv(file_path)

# Perform feature engineering

# Group by playerID and aggregate relevant statistics
grouped_df = fe_batting.groupby('playerID').agg(
    years_played=('Batting_df_yearID', lambda x: x.nunique()),
    total_games=('Batting_df_G', 'sum'),
    total_at_bats=('Batting_df_AB', 'sum'),
    total_runs=('Batting_df_R', 'sum'),
    total_hits=('Batting_df_H', 'sum'),
    total_walks=('Batting_df_BB', 'sum'),
    total_doubles=('Batting_df_2B', 'sum'),
    total_triples=('Batting_df_3B', 'sum'),
    total_home_runs=('Batting_df_HR', 'sum'),
    total_RBI=('Batting_df_RBI', 'sum'),
    total_SB=('Batting_df_SB', 'sum'),
    total_CS=('Batting_df_CS', 'sum'),
    total_BB=('Batting_df_BB', 'sum'),
    total_SO=('Batting_df_SO', 'sum'),
    total_IBB=('Batting_df_IBB', 'sum'),
    total_HBP=('Batting_df_HBP', 'sum'),
    total_SH=('Batting_df_SH', 'sum'),
    total_SF=('Batting_df_SF', 'sum'),
    total_GIDP=('Batting_df_GIDP', 'sum'),
    HOF_status=('HallOfFame_df_inducted', 'max'),
    HOF_year=('HallOfFame_df_yearid', 'max'),
    BattingPost_G=('BattingPost_df_G', 'sum'),
    BattingPost_AB=('BattingPost_df_AB', 'sum'),
    BattingPost_R=('BattingPost_df_R', 'sum'),
    BattingPost_H=('BattingPost_df_H', 'sum'),
    BattingPost_2B=('BattingPost_df_2B', 'sum'),
    BattingPost_3B=('BattingPost_df_3B', 'sum'),
    BattingPost_HR=('BattingPost_df_HR', 'sum'),
    BattingPost_RBI=('BattingPost_df_RBI', 'sum'),
    BattingPost_SB=('BattingPost_df_SB', 'sum'),
    BattingPost_CS=('BattingPost_df_CS', 'sum'),
    BattingPost_BB=('BattingPost_df_BB', 'sum'),
    BattingPost_SO=('BattingPost_df_SO', 'sum'),
    BattingPost_IBB=('BattingPost_df_IBB', 'sum'),
    BattingPost_HBP=('BattingPost_df_HBP', 'sum'),
    BattingPost_SH=('BattingPost_df_SH', 'sum'),
    BattingPost_SF=('BattingPost_df_SF', 'sum'),
    BattingPost_GIDP=('BattingPost_df_GIDP', 'sum')
).reset_index()

grouped_df['total_singles'] = grouped_df['total_hits'] - (grouped_df['total_doubles'] + grouped_df['total_triples'] + grouped_df['total_home_runs'])

# # Merge back with the original DataFrame to preserve all columns
# merge_df = pd.merge(grouped_df, grouped_df, on='playerID', how='left')

# Calculate new features
grouped_df['batting_avg'] = grouped_df['total_hits'] / grouped_df['total_at_bats']
grouped_df['OBP'] = (grouped_df['total_hits'] + grouped_df['total_walks']) / (grouped_df['total_at_bats'] + grouped_df['total_walks'])
grouped_df['SLG'] = (
    (grouped_df['total_singles'] - grouped_df['total_doubles'] - grouped_df['total_triples'] - grouped_df['total_home_runs']) +
    (2 * grouped_df['total_doubles']) +
    (3 * grouped_df['total_triples']) +
    (4 * grouped_df['total_home_runs'])
) / grouped_df['total_at_bats']
grouped_df['OPS'] = grouped_df['OBP'] + grouped_df['SLG']

# grouped_df.to_csv('Award_Batting_HOF_FE_csv.csv', index=False)
# grouped_df.to_feather('Award_Batting_HOF_FE.csv')

grouped_df.head()
grouped_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Data columns (total 44 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   playerID          499 non-null    object 
 1   years_played      499 non-null    int64  
 2   total_games       499 non-null    int64  
 3   total_at_bats     499 non-null    int64  
 4   total_runs        499 non-null    int64  
 5   total_hits        499 non-null    int64  
 6   total_walks       499 non-null    int64  
 7   total_doubles     499 non-null    int64  
 8   total_triples     499 non-null    int64  
 9   total_home_runs   499 non-null    int64  
 10  total_RBI         499 non-null    int64  
 11  total_SB          499 non-null    int64  
 12  total_CS          499 non-null    int64  
 13  total_BB          499 non-null    int64  
 14  total_SO          499 non-null    int64  
 15  total_IBB         499 non-null    int64  
 16  total_HBP         499 non-null    int64  
 1